In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, MessageGraph,START,END
from langgraph.graph.message import add_messages

from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal,TypedDict, List

from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3)
    response = llm.invoke("Are you ready to help me")
    print(response.content)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    

Yes, I'm ready to help you.  Please tell me what you need.


In [3]:
class State(TypedDict):
    messages : Annotated[List[str], add_messages]


In [6]:
def chat(state: State):
    system = """ You are an AI assistant for answer the question as much as possible. Your tone must be polite"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system' , system),
            ('human', '{messages}')
        ]
    )
    chain = prompt | llm
    response = chain.invoke({'messages': state['messages']})
    return {'messages':response}

In [7]:
state = State()
state['messages'] = 'what is hello in france'

r = chat(state)

In [8]:
r

{'messages': AIMessage(content='The French word for "hello" is **bonjour**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--04368fb0-f000-44f2-a1ca-52e923e845ee-0', usage_metadata={'input_tokens': 24, 'output_tokens': 12, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})}

In [10]:
workflow = StateGraph(State)
workflow.add_node('chat',chat)
workflow.add_edge(START, 'chat')
workflow.add_edge('chat', END)
graph = workflow.compile()

In [11]:
graph.invoke({'messages': 'what is hello in Thailand'})

{'messages': [HumanMessage(content='what is hello in Thailand', additional_kwargs={}, response_metadata={}, id='e0c8b019-8231-4134-89d5-95181124e691'),
  AIMessage(content='สวัสดี (s̄wàsdī) is the most common way to say "hello" in Thailand.  It\'s pronounced roughly as "sà-wàt-dee".', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--0f3e0d0a-fd26-4d83-a504-2f7fbcccccc4-0', usage_metadata={'input_tokens': 78, 'output_tokens': 42, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}})]}